# Setup

In [ ]:
import numpy as np
import pandas as pd
import os
import copy
from training_loops import train_ef_IM_bi
from tqdm import tqdm
import random
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# NCL to PCL with BiEFLSTM and IEMOCAP

## Data

In [ ]:
#Multimodal Data
X_train = np.load("../data/iemocap/X_train.npy")
X_val = np.load("../data/iemocap/X_val.npy")
X_test = np.load("../data/iemocap/X_test.npy")

#Creating only text dataset (first 300 features are only language)
X_train_text = X_train.copy()
X_train_text[:,:,300:] = 0.0

X_val_text = X_val.copy()
X_val_text[:,:,300:] = 0.0

X_test_text = X_test.copy()
X_test_text[:,:,300:] = 0.0

#Labels
Y_train = np.load("../data/iemocap/Y_train.npy")
Y_val = np.load("../data/iemocap/Y_val.npy")
Y_test = np.load("../data/iemocap/Y_test.npy")

#Total Classes in IEMOCAP Dataset after dropping xxx and oth
total_classes = 9

## Hyperparameters

In [ ]:
config = dict()
config["batchsize"] = 15 *2
config["num_epochs"] = 40
config["lr"] = 0.0001
config["h"] = 128
config["drop"] = 0.5
config['input_dims'] = [300,80,310]

## Training and Testing 

In [ ]:
#Training and Testing across Multiple seeds
total_seeds = 5
for i in range(total_seeds):
    set_seed(i)

    #Training across different Modality drop rates
    modality_drop = [[0,0,0],[.15,.15,0],[.4,.4, 0],[.6,.6, 0], [.8,.8,0], [.9,.9,0], [.95,.95,0]]
    res = []
    cm_res = []
    for mod_drop_probs in tqdm(modality_drop):

        a_d = mod_drop_probs[0]
        v_d = mod_drop_probs[1]
        l_d = mod_drop_probs[2]

        results = train_ef_IM_bi(X_train, Y_train, X_val_text, Y_val, X_test_text, Y_test, config, total_classes, a_d,v_d,l_d)
        res.append(results[:-1])
        cm_res.append(results[-1])
    
    #Now running Unimodal Version
    results = train_ef_IM_bi(X_train_text, Y_train, X_val_text, Y_val, X_test_text, Y_test, config, total_classes, 0,0,0)
    #Since this is unimodal set N/A to mod dropout
    results[0] = 'N/A'
    results[1] = 'N/A'
    results[2] = 'N/A'
    res.append(results[:-1])
    cm_res.append(results[-1])

    cols = ['audio_dropout', 'language_dropout', 'video_dropout', 'acc', 'loss', 'f_score']
    result_df = pd.DataFrame(res, columns = cols)
    result_df.to_csv('results/co_learning_bilstm_im/elstm_ie'+ str(i) +'.csv', sep = '\t')
    np.save('results/co_learning_bilstm_im/cm_res', cm_res)
    /content/drive/MyDrive/PCL to NCL/results/co_learning_bilstm_im
    

  0%|          | 0/7 [00:00<?, ?it/s]

0 2.102349439559624 2.0061888694763184 saving model
1 1.9390124758781746 1.9049832820892334 saving model
2 1.8463235361534251 1.8897647857666016 saving model
3 1.827397456643177 1.8689473867416382 saving model
4 1.7826323383732845 1.849735140800476 saving model
5 1.7638842578519855 1.8202273845672607 saving model


 14%|█▍        | 1/7 [00:53<05:22, 53.73s/it]

[[  0   5   0   0   0   0   0   0]
 [  0 192   0   0   0   0   0   0]
 [  0 130   0   0   0   0   0   0]
 [  0 202   0   1   0   0   0   0]
 [  0 105   0   0   0   0   0   0]
 [  0  15   0   0   0   0   0   0]
 [  0  65   0   0   0   0   0   0]
 [  0 145   0   0   0   0   0   0]]
[0, 0, 0, 0.22441860465116278, tensor(1.8352, device='cuda:0'), 0.36495]
0 2.1771811177158913 2.0300042629241943 saving model
1 1.9708195827160662 1.9027267694473267 saving model
2 1.9005930674703497 1.839236855506897 saving model
5 1.8125872618970815 1.83681058883667 saving model
7 1.7851834108954983 1.8182590007781982 saving model
17 1.6889120312462076 1.7969210147857666 saving model
19 1.6744316230740464 1.788240909576416 saving model
21 1.6740928107534934 1.78410005569458 saving model
22 1.6593992250007497 1.759250521659851 saving model
30 1.6272875628276178 1.7513902187347412 saving model
32 1.619976976461578 1.7395944595336914 saving model
34 1.61105119484907 1.7208729982376099 saving model
35 1.59339694

 29%|██▊       | 2/7 [01:38<04:02, 48.49s/it]

...

[[  0   3   0   1   1   0   0   0]
 [  0 115   6  38  14   0   5  14]
 [  0  25  46  46   5   0   1   7]
 [  0  65  19  91   3   0   1  24]
 [  0  29   5   8  49   0   4  10]
 [  0   6   0   5   3   0   0   1]
 [  0  25   4   5  13   0  12   6]
 [  0  30   1  22  14   0   1  77]]
[0.9, 0, 0.9, 0.45348837209302323, tensor(1.4703, device='cuda:0'), 0.46413]
0 2.1977152901086194 2.0411267280578613 saving model
1 1.967715062593159 1.8865708112716675 saving model
2 1.8688163053222566 1.8769139051437378 saving model
3 1.813032917809068 1.8280316591262817 saving model
4 1.7627839247385662 1.74504554271698 saving model
5 1.7340919330106144 1.7443654537200928 saving model
6 1.7035900326500162 1.6996783018112183 saving model
8 1.641936578945807 1.6860140562057495 saving model
9 1.6012078628205417 1.6439311504364014 saving model
11 1.5618719740917808 1.6386281251907349 saving model
12 1.5501637807366444 1.6128404140472412 saving model
13 1.5140586847450301 1.6025043725967407 saving model
15 1.471

100%|██████████| 7/7 [05:24<00:00, 46.42s/it]

39 1.1017575344147041 1.4687795639038086 saving model
[[  0   1   2   2   0   0   0   0]
 [  0 125   3  35  10   0   6  13]
 [  0  31  48  37   4   0   0  10]
 [  0  68  18  91   1   1   0  24]
 [  0  38   2   7  41   0  10   7]
 [  0  11   0   2   1   1   0   0]
 [  0  28   6   5   9   0  14   3]
 [  0  49   2  19  10   0   3  62]]
[0.95, 0, 0.95, 0.4441860465116279, tensor(1.4919, device='cuda:0'), 0.45152]


0 2.151294437765378 1.978798270225525 saving model
1 1.9542620230836478 1.8768199682235718 saving model
2 1.8665909732294361 1.820921540260315 saving model
3 1.8139185982140882 1.7779821157455444 saving model
4 1.7701582608864321 1.7423573732376099 saving model
5 1.7246498726961905 1.7109644412994385 saving model
6 1.6861085982350579 1.6923151016235352 saving model
7 1.6566779962060048 1.670168399810791 saving model
8 1.6280734030126829 1.6506869792938232 saving model
9 1.589195606304191 1.6381932497024536 saving model
10 1.56959343026256 1.620481252670288 saving model
11 1.5402481827819556 1.6138042211532593 saving model
12 1.511077334309182 1.6128861904144287 saving model
14 1.464559977514702 1.5729233026504517 saving model
17 1.392521279597143 1.5490095615386963 saving model
23 1.2810146766796446 1.5365725755691528 saving model
24 1.2584563169563026 1.5345901250839233 saving model
27 1.1949248380131192 1.5334619283676147 saving model
29 1.1691835146201284 1.510542869567871 saving mo

FileNotFoundError: ignored